In [1]:
!pip install --upgrade jupyterlab

   ---------------------------------------- 0.0/11.7 MB ? eta -:--:--
    --------------------------------------- 0.3/11.7 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.7 MB 2.6 MB/s eta 0:00:05
   -------- ------------------------------- 2.4/11.7 MB 4.6 MB/s eta 0:00:03
   --------------- ------------------------ 4.5/11.7 MB 6.7 MB/s eta 0:00:02
   ----------------------- ---------------- 6.8/11.7 MB 7.6 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.7 MB 8.0 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.7 MB 8.7 MB/s eta 0:00:01
   ---------------------------------------- 11.7/11.7 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: jupyterlab
    Found existing installation: jupyterlab 4.3.5
    Uninstalling jupyterlab-4.3.5:


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'd:\\codes\\hackathon\\llm_engineering\\llms\\scripts\\jupyter-lab.exe'
Check the permissions.



In [13]:
# imports
import os
import requests
from bs4 import BeautifulSoup
from typing import List
import gradio as gr
import ollama

In [14]:
system_message = "You are a helpful assistant"
MODEL="llama3.2:latest"

In [15]:
def message_llama(propmpt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": propmpt}
    ]
    completion = ollama.chat(model=MODEL, messages=messages)
    return completion.message.content

In [16]:
message_llama("What is today's weather?")

"I'm happy to help you, but I'm a large language model, I don't have real-time access to current weather conditions. However, I can suggest some ways for you to find out the current weather:\n\n1. **Check online weather websites**: You can check websites like AccuWeather, Weather.com, or the National Weather Service (NWS) for up-to-date weather forecasts.\n2. **Use a mobile app**: Download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get current weather conditions and forecasts.\n3. **Tune into local news**: Watch local news or listen to local radio stations to get the latest weather forecast.\n\nPlease note that I can provide general information about different types of weather or help with other topics if you'd like!"

In [17]:
# here's a simple function
def shout(text):
    print(f"Shout has been called with input: {text}")
    return text.upper()

In [18]:
shout("hello")

Shout has been called with input: hello


'HELLO'

In [19]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [20]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [21]:
# Inputs and Outputs
view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=6)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [22]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_llama,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    title="Llama Chat",
    description="A simple chatbot that responds in markdown."
)

view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [23]:
# Let's create a call that streams back results

def stream_llama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    completion = ollama.chat(model=MODEL, messages=messages, stream=True)
    result = ""
    for message in completion:
        result += message.message.content
        yield result

In [24]:
view = gr.Interface(
    fn=stream_llama,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    title="Llama Chat",
    description="A simple chatbot that responds in markdown."
)

view.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [25]:
def stream_model(prompt, model):
    if model == "llama3.2:latest":
        result = stream_llama(prompt)
    else:
        raise ValueError("Invalid model")
    yield from result

In [26]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [27]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:", lines=6), gr.Dropdown(["llama3.2:latest"], label="Model:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    title="Llama Chat",
    js = force_dark_mode,
    description="A simple chatbot that responds in markdown."
)

view.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [31]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [32]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [33]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="llama3.2:latest":
        result = stream_llama(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [34]:
stream_brochure("ED", "https://edwarddonner.com", "llama3.2:latest")

<generator object stream_brochure at 0x000001961162D030>

In [35]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["llama3.2:latest"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
